http://www.sports-reference.com/cbb/postseason/1939-ncaa.html

In [50]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import element
from time import sleep
import pandas as pd


def get_page_data(year):
    url = 'http://www.sports-reference.com/cbb/postseason/'+str(year)+'-ncaa.html'
    page = urlopen(url)
    return BeautifulSoup(page, 'html.parser')

def tidy_location(locStr):
    if locStr.startswith('at '):
        return locStr[len('at '):]

def get_bracket():    
    years = range(2017, 1985, -1)
    #TODO fix years earlier than 1985
    bracket = []
    for year in years: 
        soup = get_page_data(year)
        region_tags = soup.find_all("div", attrs={"data-controls": "#brackets"})[0].select('div a')        
        for region_tag in region_tags:        
            region = str(region_tag.string).lower()
            if region is not 'final four':            
                search = {'id': region}        
                region_div = soup.find('div',**search)
                if region_div is not None:
                    region_first_round = region_div.div.div                
                    for elem in region_first_round.contents:
                        if isinstance(elem, element.Tag ):
                            seeds_loc = elem.find_all('span')
                            #if len(seeds_loc) == 2:
                            seed1 = seeds_loc[0].string
                            seed2 = seeds_loc[1].string
                            currLoc = tidy_location(seeds_loc[2].string)        
                            alist = elem.select('div a')
                            team1 = alist[0].string
                            team2 = alist[2].string        
                            bracketRow = [year, region, seed1, team1, currLoc, seed2, team2]
                            bracket.append(bracketRow) 
        sleep(0.5)
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Seed', 'Team Name','Location', 'Oppenent Seed', 'Opponent']) 

df = get_bracket()
df

NotImplementedError: Only the following pseudo-classes are implemented: nth-of-type.

[]